In [ ]:

%pip install pandas
%pip install numpy
%pip install scanpy
%pip install anndata


^C
Note: you may need to restart the kernel to use updated packages.




Import packages

In [ ]:
#general
import os
import glob
import pathlib
import tarfile
import numpy as np
import scanpy as sc
import pandas as pd

import anndata as ad
from anndata.experimental.multi_files import AnnCollection




In [ ]:
import pathlib
current_directory = pathlib.Path.cwd()
module_directory = current_directory.parent.parent
input_path = module_directory / "input"
zip_path = input_path / "biostudiesarrayE-MTAB-14039"
extracted_path = input_path / "extracted_data"
hd5a_path = input_path / 'hd5a_files'
scrna_metadata_path = input_path /"scRNA_metadata"


In [ ]:
import shutil

total, used, free = shutil.disk_usage("/")
print(f"Total: {total // (2**30)} GiB")
print(f"Used: {used // (2**30)} GiB")
print(f"Free: {free // (2**30)} GiB")

In [ ]:

# Function to extract tar.gz files
def extract_tar_gz(tar_gz_path, extract_dir):
    with tarfile.open(tar_gz_path, "r:gz") as tar:
        tar.extractall(path=extract_dir)

# Directory containing your datasets
  # Folder to save the extracted files
os.makedirs(extracted_path, exist_ok=True)

# List all top-level files in the input directory
tar_gz_files = [f for f in os.listdir(zip_path) if f.endswith('.tar.gz')]

# Extract tar.gz files
for tar_gz_file in tar_gz_files:
    tar_gz_path = os.path.join(zip_path, tar_gz_file)
    
    # Extract the tar.gz file to the extracted directory
    extract_tar_gz(tar_gz_path, extracted_path)


In [ ]:

# Directory where the files have been extracted
# List to store directory names under the 'work' directory
dataset_dirs = []

# Traverse the 'work' directory to find subdirectories
for root, dirs, files in os.walk(extracted_path):
    # Only consider directories directly under the 'work' directory
    if "work" in root:
        for dir_name in dirs:
            path = os.path.join(root, dir_name)
            # Append the directory name under 'work'
            dataset_dirs.append(path)

# Now dataset_dirs contains the names of all subdirectories under 'work'
print("Dataset directories under 'work':", dataset_dirs)


In [ ]:
print(dataset_dirs)


Docs: https://anndata.readthedocs.io/en/latest/tutorials/notebooks/anncollection.html
https://ccbskillssem.github.io/assets/scvi_notebook.html

warning: "If you use join='outer' this fills 0s for sparse data when variables are absent in a batch. Use this with care. Dense data is filled with NaN."

https://discourse.scverse.org/t/how-to-concatenate-anndata-properly/887/2
strongly recommends datacleaning each dataset before concatenating -- assuming that they did it for us

In [ ]:

if not os.path.exists(hd5a_path):
    os.makedirs(hd5a_path) 
combined_adatas_dict = {}
for directory in dataset_dirs:
   adata = sc.read_10x_mtx(directory)
    adata.X = adata.X.tocsr()
    #dir_name = directory.split("\\")[-1]
    dir_name = os.path.basename(directory) #platform independent
    h5ad_filename = f"{dir_name}.h5ad"

    h5ad_path = os.path.join(hd5a_path, h5ad_filename)
   combined_adata.write(h5ad_path)
    adatas_dict[dir_name] = h5ad_path


In [ ]:
print(adatas_dict)

In [ ]:

combined_on_disk_name = scrna_output/'data_combined_on_disk.h5ad'
#os.remove(combined_on_disk_name)
ad.experimental.concat_on_disk(
    adatas_dict,          # List of paths to the individual .h5ad files
    combined_on_disk_name,  # Path to the combined output file
    label='dataset'      # Optional: label for the datasets
)